# A notebook to illustrate using the db2 data pull from within the IPython Notebook
Need to define the password file where I've stored the password for my access to the database

In [ ]:
!hadoop fs -ls '/user/kesj/config'

### Setup parameters

In [ ]:
nfile = '/user/kesj/config/.myp'

#alternatively can use relative path: nfile = 'config/e.pswd'
# to use local File space you must preface the file name with file:
user = 'kesj'
#dbip = 'jdbc:db2://10.96.37.166:60100/FDW2P'
dbip= 'jdbc:db2://br95qc60.opr.test.statefarm.org:60100/fdw2f'

### connect to the database and list the databases present

In [ ]:
dbList = !sqoop list-databases --connect {dbip} --username {user} --password-file {nfile}
dbList=dbList[4:]
print "There are {0} databases (schemas) within this DB".format(len(dbList))

In [ ]:
test_dbs = [a for a in dbList if a.startswith('FDWDRVDECSCF')]

In [ ]:
test_dbs

### List all the tables in it

In [ ]:
tblList = !sqoop list-tables --connect {dbip} --username {user} --password-file {nfile}
tblList = tblList[5:]
print "There are {0} tables in these schemas.".format(len(tblList))

In [ ]:
[a for a in tblList if a.startswith('SNA')]

### define the target directory

In [ ]:
tgt_dir = '/user/kesj/sqp_test_clm_2'

### see about pulling all of one of the tables present in one of the databases
* start with FDWDRVDECSCF_263.SNA_ISO_VEH

In [ ]:
### define a table name
my_tbl = 'FDWDRVDECSCF_263.SNA_ISO_VEH'

In [ ]:
# define table name
!sqoop import --connect {dbip} --username {user} --password-file {nfile} --table {my_tbl} -m 1 --target-dir {tgt_dir}

## check to make sure the data was pulled

In [ ]:
!hdfs dfs -ls {tgt_dir}

### I previously have used a couple of additional options:
1. use a spltkey
2. use a query clause and where clause
3. use a specific encoding for the data as it lives in db2. (doesn't seem to work now)

### I think you will want to use a flag to drop this into a hive table directly

--hive-import

In [ ]:
test_dbs

In [ ]:
spltkey='CLM_ID'
wclause = '"PRTPT_ID>3"'
qclause = '"SELECT * FROM FDWDRVDECSCF_950.SNA_CLM_PRTPT_EML_RLTN WHERE PRTPT_ID == 1 and \$CONDITIONS"'

In [ ]:
tgt_dir2 = '/user/kesj/sna_clm_3'

## try  this with a query clause

In [ ]:
#!sqoop import -Ddb2.jcc.charsetDecoderEncoder=3  --connect {dbip} --driver com.ibm.db2.jcc.DB2Driver --username kesj --password-file {nfile}  --query {qclause} --fields-terminated-by "\t" --target-dir {tgt_dir2} --split-by {spltkey} #--hive-import
!sqoop import --driver com.ibm.db2.jcc.DB2Driver --connect {dbip}  --username kesj --password-file {nfile}  --query {qclause} --fields-terminated-by "\t" --target-dir {tgt_dir2} --split-by {spltkey} --hive-import --hive-table sna_950_prtpt_eml